In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.cm as cm
from matplotlib import pyplot as plt 
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, Flatten, Dense, merge, Reshape, Permute, Multiply
from keras.layers import Input
from keras import backend as K
from keras.engine.topology import Layer

# packages for learning from crowds
from crowd_layer.crowd_layers import CrowdsClassification, MaskedMultiCrossEntropy
from crowd_layer.crowd_aggregators import CrowdsCategoricalAggregator

# prevent tensorflow from allocating the entire GPU memory at once
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [17]:
NUM_RUNS = 30
DATA_PATH = "/Users/yangyajing/Documents/noisy_dataset/LabelMe/prepared/"
N_CLASSES = 8
BATCH_SIZE = 64
N_EPOCHS = 30
W = 0

In [3]:
def load_data(filename):
    f = open(filename, 'rb')
    data = np.load(f)
    f.close()
    return data

In [4]:
print("\nLoading train data...")

# images processed by VGG16
data_train_vgg16 = load_data(DATA_PATH+"data_train_vgg16.npy")
print(data_train_vgg16.shape)

# ground truth labels
labels_train = load_data(DATA_PATH+"labels_train.npy")
print(labels_train.shape)

# labels obtained from majority voting
labels_train_mv = load_data(DATA_PATH+"labels_train_mv.npy")
print(labels_train_mv.shape)

# labels obtained by using the approach by Dawid and Skene
labels_train_ds = load_data(DATA_PATH+"labels_train_DS.npy")
print(labels_train_ds.shape)

# data from Amazon Mechanical Turk
print("\nLoading AMT data...")
answers = load_data(DATA_PATH+"answers.npy")
print(answers.shape)
N_ANNOT = answers.shape[1]
print("\nN_CLASSES:", N_CLASSES)
print("N_ANNOT:", N_ANNOT)

# load test data
print("\nLoading test data...")

# images processed by VGG16
data_test_vgg16 = load_data(DATA_PATH+"data_test_vgg16.npy")
print(data_test_vgg16.shape)

# test labels
labels_test = load_data(DATA_PATH+"labels_test.npy")
print(labels_test.shape)


Loading train data...
(10000, 4, 4, 512)
(10000,)
(10000,)
(10000,)

Loading AMT data...
(10000, 59)

N_CLASSES: 8
N_ANNOT: 59

Loading test data...
(1188, 4, 4, 512)
(1188,)


In [5]:
def one_hot(target, n_classes):
    targets = np.array([target]).reshape(-1)
    one_hot_targets = np.eye(n_classes)[targets]
    return one_hot_targets

In [6]:
print("\nConverting to one-hot encoding...")
labels_train_bin = one_hot(labels_train, N_CLASSES)
print(labels_train_bin.shape)
labels_train_mv_bin = one_hot(labels_train_mv, N_CLASSES)
print(labels_train_mv_bin.shape)
labels_train_ds_bin = one_hot(labels_train_ds, N_CLASSES)
print(labels_train_ds_bin.shape)
labels_test_bin = one_hot(labels_test, N_CLASSES)
print(labels_test_bin.shape)

answers_bin_missings = []
for i in range(len(answers)):
    row = []
    for r in range(N_ANNOT):
        if answers[i,r] == -1:
            row.append(-1 * np.ones(N_CLASSES))
        else:
            row.append(one_hot(answers[i,r], N_CLASSES)[0,:])
    answers_bin_missings.append(row)
answers_bin_missings = np.array(answers_bin_missings).swapaxes(1,2)
answers_bin_missings.shape


Converting to one-hot encoding...
(10000, 8)
(10000, 8)
(10000, 8)
(1188, 8)


(10000, 8, 59)

In [7]:
answers_test_bin_missings = np.zeros((len(labels_test), N_CLASSES))
answers_test_bin_missings[np.arange(len(labels_test)), labels_test] = 1
answers_test_bin_missings = np.repeat(answers_test_bin_missings.reshape([len(labels_test),N_CLASSES,1]), N_ANNOT, axis=2)
answers_test_bin_missings.shape

(1188, 8, 59)

In [8]:
def eval(model,y_test):
    return dict(zip(model.metrics_names,model.evaluate(data_test_vgg16,y_test, verbose=False)))

In [9]:
APRIOR_NOISE=0.46
bias_weights = (
    np.array([np.array([np.log(1. - APRIOR_NOISE)
                        if i == j else
                        np.log(APRIOR_NOISE / (N_CLASSES - 1.))
                        for j in range(N_CLASSES)]) for i in
              range(N_CLASSES)])
    + 0.01 * np.random.random((N_CLASSES, N_CLASSES)))
# bias_weights = np.repeat(bias_weights, N_ANNOT, axis=1)
bias_weights

array([[-0.60846025, -2.7198483 , -2.71769437, -2.71264534, -2.71279536,
        -2.71974188, -2.71387189, -2.71309373],
       [-2.71853129, -0.61556844, -2.72062736, -2.7194094 , -2.71426949,
        -2.71796426, -2.72201468, -2.71358326],
       [-2.7214227 , -2.71382941, -0.61017251, -2.72226906, -2.71738096,
        -2.7153441 , -2.71540738, -2.7133386 ],
       [-2.71839219, -2.71948669, -2.71500308, -0.61173184, -2.71462957,
        -2.71607297, -2.71291541, -2.71672126],
       [-2.72219101, -2.71297304, -2.72223327, -2.7129742 , -0.61615523,
        -2.714629  , -2.72237577, -2.72186359],
       [-2.71798827, -2.71299412, -2.72198147, -2.7130777 , -2.71762434,
        -0.60953332, -2.71321924, -2.72189385],
       [-2.71732214, -2.72019996, -2.71855707, -2.71387811, -2.71251387,
        -2.7201515 , -0.61306523, -2.71801958],
       [-2.71244193, -2.71954322, -2.71818404, -2.71775781, -2.714386  ,
        -2.72104864, -2.72153473, -0.60895601]])

In [10]:
hidden_layers = Sequential()
hidden_layers.add(Flatten(input_shape=data_train_vgg16.shape[1:]))
hidden_layers.add(Dense(128, activation='relu'))
hidden_layers.add(Dropout(0.5))

train_inputs = Input(shape=(data_train_vgg16.shape[1:]))
last_hidden = hidden_layers(train_inputs)
baseline_output = Dense(N_CLASSES, activation='softmax', name='baseline')(last_hidden)
print(last_hidden)

# add crowds layer on top of the base model
# channeled_output = CrowdsClassification(N_CLASSES, N_ANNOT)(baseline_output)
channeled_output = []
for w in range(59):
    channel_matrix = [Dense(N_CLASSES,
                            activation='softmax',
                            name='dense_class%d_annot%d'%(i,w),
                            weights=[
                                W*(np.random.random((128,N_CLASSES)) - 0.5),
                                bias_weights[i]
                            ])(last_hidden)
                      for i in range(N_CLASSES)]
    channel_matrix = merge(channel_matrix, mode='concat')
    channel_matrix = Reshape((N_CLASSES, N_CLASSES))(channel_matrix)

    channeled_output_w = merge([channel_matrix, baseline_output], mode='dot', dot_axes=(1,1), name='channeled_annot%d'%w)
    channeled_output.append(channeled_output_w)

mask = np.equal(y_true[:,0,:], -1)
zer = tf.zeros_like(vec)
loss = tf.where(mask, x=zer, y=vec)

channeled_output = merge(channeled_output, mode='concat')
channeled_output = Reshape((-1, N_CLASSES))(channeled_output)
channeled_output = Permute((2,1))(channeled_output)
channeled_output = Multiply([channeled_output, mask])
print(channeled_output)

simple_model = Model(inputs=train_inputs, outputs=[channeled_output, baseline_output])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Tensor("sequential_1/dropout_1/cond/Merge:0", shape=(?, 128), dtype=float32)


/Users/yangyajing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/yangyajing/anaconda/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/yangyajing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Tensor("merge_1/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_1/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot0/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_2/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_2/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot1/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_3/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_3/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot2/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_4/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_4/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot3/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_5/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_5/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot4/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_6/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_6/

Tensor("merge_45/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_45/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot44/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_46/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_46/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot45/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_47/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_47/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot46/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_48/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_48/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot47/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_49/concat:0", shape=(?, 64), dtype=float32)
Tensor("reshape_49/Reshape:0", shape=(?, 8, 8), dtype=float32)
Tensor("channeled_annot48/Squeeze:0", shape=(?, 8), dtype=float32)
Tensor("merge_50/concat:0", shape=(?, 64), dtype=float32)
Te

/Users/yangyajing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [13]:
# instantiate specialized masked loss to handle missing answers
loss = MaskedMultiCrossEntropy().loss

# compile model with masked loss and train
simple_model.compile(optimizer='adam', 
                     loss=[loss,'categorical_crossentropy'], 
                     loss_weights=[1,0],
                     metrics=['accuracy']
                    )

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [18]:
simple_model.fit(data_train_vgg16, [answers_bin_missings, labels_train_mv_bin], epochs=N_EPOCHS, shuffle=True, batch_size=BATCH_SIZE, verbose=1)

Epoch 1/30
10000/10000 [==============================] - 13s - loss: 0.0596 - permute_1_loss: 0.0596 - baseline_loss: 4.4838 - permute_1_acc: 0.0118 - baseline_acc: 0.6873    
Epoch 2/30
10000/10000 [==============================] - 12s - loss: 0.0594 - permute_1_loss: 0.0594 - baseline_loss: 4.4908 - permute_1_acc: 0.0123 - baseline_acc: 0.6889    
Epoch 3/30
10000/10000 [==============================] - 12s - loss: 0.0592 - permute_1_loss: 0.0592 - baseline_loss: 4.4702 - permute_1_acc: 0.0132 - baseline_acc: 0.6922    
Epoch 4/30
10000/10000 [==============================] - 12s - loss: 0.0588 - permute_1_loss: 0.0588 - baseline_loss: 4.4654 - permute_1_acc: 0.0137 - baseline_acc: 0.6936    
Epoch 5/30
10000/10000 [==============================] - 13s - loss: 0.0587 - permute_1_loss: 0.0587 - baseline_loss: 4.4136 - permute_1_acc: 0.0138 - baseline_acc: 0.6930    - ETA: 7s - loss: 0.0584 - p
Epoch 6/30
10000/10000 [==============================] - 13s - loss: 0.0586 - permute_

In [19]:
eval(simple_model,y_test=[answers_test_bin_missings,labels_test_bin])

{'baseline_acc': 0.6978114478114478,
 'baseline_loss': 3.783887464992125,
 'loss': 1.6680947823155208,
 'permute_1_acc': 0.049663299663299666,
 'permute_1_loss': 1.6680947823155208}